# Descriptors #

*Descriptors* are [defined](https://docs.python.org/3/reference/datamodel.html#descriptors) in the Python language reference in a very abstract way: it is not clear what they really are for. In fact, they are quite a fundamental feature of the language, it’s just that you often use them without even being aware of it.

**Every function is automatically a descriptor**. When you define a function, it initially becomes a *non-data descriptor*: it is an object with a `__get__()` method, but none of the other special descriptor methods.

In [ ]:
def testfunc() :
    pass
#end testfunc

for attr in ("__get__", "__set__", "__delete__", "__set_name__") :
    print("has %s? %s" % (attr, hasattr(testfunc, attr)))
#end for

Of course, this can be changed by redefining the attributes of the function object. Thus, the [`property()`](https://docs.python.org/3/library/functions.html#property) builtin turns a function into a data descriptor by attaching `__set__()` and `__delete__()` methods. Note that it has to do this even for read-only properties; if no setter is defined for the property, then `__set__()` simply raises an exception saying the property is read-only.

### When Do Descriptors Come Into Effect? ###

Descriptors come into play when you access them through an *instance* of a class. When you use a construct like

    «inst».«attr»

where «inst» is some expression, and «attr» is some name, then Python goes through a procedure something like this:
* Is «attr» the name of an instance member of object «inst»? If so, then access the value of that instance member.
* If not, is «inst» an instance of some class «class», and is «attr» the name of a member of that class? If so:
* Is the value of «class».«attr» a descriptor? If not, then access the value of that class member.
* But if it is, and the access is not the LHS of an assignment or a `del` statement, call its `__get__()` method.
* If the access is the LHS of an assignment and the descriptor has a `__set__()` method (it is a data descriptor), call that.
* If the access is in a `del` statement and the descriptor has a `__delete__()` method (data descriptor), call that.
* Otherwise, for an assignment to a non-data descriptor, create an instance member with that name and set it to the value of the RHS. `del` of a nonexistent instance member is an error.

In the following, `TestClass` is a test class that uses some specially-constructed descriptor classes to demonstrate the behaviour:

In [ ]:
class NonDataDescriptor :
    "constructs a dummy non-data descriptor with an identifying name."

    def __init__(self, name) :
        self.name = name
    #end __init__
    
    def __get__(self, *args) :
        print("called nondata “%s” get(%s)" % (self.name, repr(args)))
    #end __get__
    
#end NonDataDescriptor

class DataDescriptor :
    "constructs a dummy data descriptor with an identifying name."

    def __init__(self, name) :
        self.name = name
    #end __init__

    def __get__(self, *args) :
        print("called data “%s” get(%s)" % (self.name, repr(args)))
    #end __get__

    def __set__(self, *args) :
        print("called data “%s” set(%s)" % (self.name, repr(args)))
    #end __set__
    
#end DataDescriptor

class TestClass :
    nondata = NonDataDescriptor("meth")
    data = DataDescriptor("prop")
#end TestClass

(It may seem a bit recursive to illustrate the behaviour of methods of class instances by calling methods of yet more class instances, but hey, it works.)

Now, if you create an instance of `TestClass` and try accessing the class attributes through this instance, Python will automatically invoke the `__get__()` methods on those attribute objects:

In [ ]:
t = TestClass()
t.nondata
t.data

Notice how the `__get__()` methods automatically get called; normally you need parentheses somewhere to indicate a function call, but here there are no parentheses to trigger those calls at all.

When you try assigning to those attributes on the class instance, the non-data descriptor will be overridden by creating an attribute of that name on the instance. However, the data descriptor *cannot* be overridden in this way: instead, its `__set__()` method will be invoked, passing the value that you attempt to assign:

In [ ]:
t.nondata = "something new"
t.nondata

In [ ]:
t.data = "something new"
t.data

As you can see, after the assignment to `t.nondata`, the `NonDataDescriptor.__get__()` method is no longer being invoked, while both references to `t.data` are passed through to appropriate methods of `DataDescriptor`. Thus, you see output generated from method calls in the latter two lines, but not the first two.

#### What Do These Methods Do? ####

The default `__get__()` method for a function constructs a *method* object. When called, this invokes the original function, prepending the class instance onto the argument list. In other words,

    «instance».«func»(«args» ...)

gets turned into

    type(«instance»).«func».__get__(«func», «instance», «class»)(«args» ...)

which gets turned into

    type(«instance»).«func»(«instance», «args» ...)

Note that a new method object gets created on *every invocation of `__get__()`*:


In [ ]:
import types

class TestClass1 :

    def test_method(self) :
        pass
    #end test_method

#end TestClass

test_inst = TestClass1()

print("function is of FunctionType? ", isinstance(TestClass1.test_method, types.FunctionType))
print("instance method is of FunctionType? ", isinstance(test_inst.test_method, types.FunctionType))
print("instance method is of MethodType? ", isinstance(test_inst.test_method, types.MethodType))
print("same method object each time? ", test_inst.test_method is test_inst.test_method)
print("equal method object each time? ", test_inst.test_method == test_inst.test_method)

But each method object has a `__func__` attribute which refers back to the original function definition:

In [ ]:
print("method objects have same __func__?", test_inst.test_method.__func__ is test_inst.test_method.__func__)

So what does the default `__get__()` method of a function object do? Let us see if we can synthesize its behaviour by hand. Here we define an `InstanceMethodMaker` class which tries to duplicate the behaviour that Python provides for functions anyway:

In [ ]:
class InstanceMethodMaker :

    def __init__(self, func) :
        self.__func__ = func
    #end __init__

    def __get__(self, instance, owner) :

        def callfunc(*args, **kwargs) :
            return \
                self.__func__(instance, *args, **kwargs)
        #end callfunc

    #begin __get__
        return \
            callfunc
    #end ___get__

#end InstanceMethodMaker

class MyClass :
    pass # no methods defined here!
#end MyClass

def my_method(*args) :
    print("my_method got called with args %s" % repr(args))
#end my_method

MyClass.instance_method = InstanceMethodMaker(my_method)
my_instance = MyClass()

Referencing the instance method causes our `__get__()` function to be invoked, which returns a reference to its inner `callfunc` function:

In [ ]:
my_instance.instance_method

And when we call this, it implements the actual dispatching to the function as an instance method:

In [ ]:
my_instance.instance_method("hi there")

Python’s [`classmethod`](https://docs.python.org/3/library/functions.html#classmethod) function changes the method definition to pass the class rather than  the instance to the method function. Let’s see how we would emulate this:

In [ ]:
class ClassMethodMaker :

    def __init__(self, func) :
        self.__func__ = func
    #end __init__

    def __get__(self, instance, owner) :

        def callfunc(*args, **kwargs) :
            return \
                self.__func__(owner, *args, **kwargs)
        #end callfunc

    #begin __get__
        return \
            callfunc
    #end ___get__

#end ClassMethodMaker

MyClass.class_method = ClassMethodMaker(my_method)

In [ ]:
my_instance.class_method

In [ ]:
my_instance.class_method("hi there")

Of course, accessing it via the class works too:

In [ ]:
MyClass.class_method

In [ ]:
MyClass.class_method("hi there")

The behaviour of the [`staticmethod()`](https://docs.python.org/3/library/functions.html#staticmethod) decorator is simplest of all: its `__get__()` method simply ignores the passed class and instance arguments, and returns the original function.

In [ ]:
def orig_teststaticmethod() :
    pass
#end orig_teststaticmethod
teststaticmethod = staticmethod(orig_teststaticmethod)

In [ ]:
print(teststaticmethod.__get__("dummy") is orig_teststaticmethod)
# returns True

OK, how about trying to emulate the behaviour of the `property()` built-in function?

Here is a (partial) implementation of equivalent functionality (missing out the deletor and docstring):

In [ ]:
class MyProp :

    def __init__(self, getx = None, setx = None) :
        self._getx = getx
        self._setx = setx
    #end __init__

    def __get__(self, inst, owner) :
        if self._getx == None :
            raise RuntimeError("non-readable property")
        #end if
        return \
            self._getx(inst)
    #end __get__

    def __set__(self, inst, value) :
        if self._setx == None :
            raise RuntimeError("non-writable property")
        #end if
        self._setx(inst, value)
    #end __set__

    def setter(self, setx) :
        self._setx = setx
        return \
            self
    #end setter

#end MyProp

Here is a use of `MyProp`. Note that the getter and setter deliberately modify the values set and returned, just to prove that they really are being called:

In [ ]:
class MyPropTest :

    def __init__(self) :
        self._prop = ""
    #end __init__

    @MyProp
    def prop(self) :
        return \
            "get " + self._prop
    #end prop

    @prop.setter
    def prop(self, value) :
        self._prop = "set " + value
    #end prop

#end MyPropTest

In [ ]:
m = MyPropTest()
m.prop = "newval"
print(m.prop)
m.prop = "new " + m.prop
print(m.prop)

Directly accessing the value of the instance variable shows the difference:

In [ ]:
m._prop

## Summary ##

It is easy to implement the behaviour of the `classmethod()`, `staticmethod()` and `property()` functions for yourself in pure Python. The only reasons for having them built into the language might be
* Efficiency of not being implemented in interpreted Python.
* They are considered such a fundamental part of the language that they should not even be part of some standard library module, but are built-in functions.

## Further Reading ##

It is instructive to read, in [Guido van Rossum’s own words](http://python-history.blogspot.com/2010/06/inside-story-on-new-style-classes.html), the rationale behind the concept of descriptors and the whole “new-style” class architecture. Those with long memories, stretching back to Python 2.*x*, may remember the original “old-style” classes, which had some deficiencies. In Python 3.*x*, *all* classes are “new-style”.